# Multi-Step Financial & Search Agent

This notebook implements a multi-step agent using **LiquidAI/LFM2.5-1.2B-Instruct**.
The agent is designed to:
1. Perform arithmetic (+, -, *, /).
2. Fetch latest crypto and stock prices via `yfinance`.
3. Search for latest news via `Tavily`.

### Agent Flow:
1. **Router**: Analyzes the query and identifies necessary tool calls (JSON output).
2. **Executor**: Executes identified tool calls and retrieves results.
3. **Synthesizer**: Combines the results into a final cohesive answer.

## 1. Install Dependencies

In [ ]:
!pip install -q transformers torch yfinance tavily-python accelerate

## 2. Configuration & Model Loading

### 2.1 API Keys

In [ ]:
import os
from getpass import getpass

# Set API Keys
try:
    from google.colab import userdata
    HF_TOKEN = userdata.get('HF_TOKEN')
    TAVILY_API_KEY = userdata.get('TAVILY_API_KEY')
except:
    HF_TOKEN = os.getenv('HF_TOKEN')
    TAVILY_API_KEY = os.getenv('TAVILY_API_KEY')

if not HF_TOKEN:
    HF_TOKEN = getpass("Enter your Hugging Face Token (HF_TOKEN): ")

if not TAVILY_API_KEY:
    TAVILY_API_KEY = getpass("Enter your Tavily API Key (TAVILY_API_KEY): ")

# Export to environment for tools to use if needed
os.environ['HF_TOKEN'] = HF_TOKEN
os.environ['TAVILY_API_KEY'] = TAVILY_API_KEY

### 2.2 Model Loading

In [ ]:
import torch
import json
from transformers import AutoModelForCausalLM, AutoTokenizer

# Robust Device Configuration
if torch.cuda.is_available():
    try:
        major, minor = torch.cuda.get_device_capability()
        if major < 7:
            print(f"GPU Capability {major}.{minor} is too low for current PyTorch. Falling back to CPU.")
            DEVICE = "cpu"
        else:
            DEVICE = "cuda"
    except:
        DEVICE = "cpu"
else:
    DEVICE = "cpu"

TORCH_DTYPE = torch.float16 if DEVICE == "cuda" else torch.float32
# Switched to Instruct version for more consistent direct outputs
MODEL_NAME = "LiquidAI/LFM2.5-1.2B-Instruct"

print(f"Loading model {MODEL_NAME} on {DEVICE}...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, token=HF_TOKEN)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=TORCH_DTYPE,
    device_map="auto" if DEVICE == "cuda" else None,
    token=HF_TOKEN
)
if DEVICE == "cpu":
    model = model.to("cpu")

def generate(prompt, max_new_tokens=500):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=max_new_tokens, do_sample=False)
    return tokenizer.decode(outputs[0][len(inputs["input_ids"][0]):], skip_special_tokens=True)

Loading model LiquidAI/LFM2.5-1.2B-Instruct on cuda...


## 3. Tool Definitions

In [ ]:
import yfinance as yf
from tavily import TavilyClient

def arithmetic_tool(op, a, b):
    """Performs basic arithmetic operations."""
    try:
        a, b = float(a), float(b)
        if op == 'add': return a + b
        if op == 'subtract': return a - b
        if op == 'multiply': return a * b
        if op == 'divide': return a / b if b != 0 else "Error: Division by zero"
    except Exception as e:
        return f"Arithmetic Error: {e}"
    return "Invalid operation"

def price_tool(symbol):
    """Fetches the latest price for a stock or crypto symbol."""
    try:
        # Auto-correct common crypto symbols
        crypto_majors = ["BTC", "ETH", "SOL", "BNB", "XRP", "ADA", "DOGE"]
        if symbol.upper() in crypto_majors:
            symbol = f"{symbol.upper()}-USD"
        
        # Auto-correct common stock naming errors
        if symbol.upper() == "NVIDIA": symbol = "NVDA"
        if symbol.upper() == "TESLA": symbol = "TSLA"
        if symbol.upper() == "GOOGLE": symbol = "GOOGL"

        ticker = yf.Ticker(symbol)
        hist = ticker.history(period="1d")
        if not hist.empty:
            return float(hist['Close'].iloc[-1])
        return f"Could not find price for {symbol}. Try a specific ticker like NVDA or BTC-USD."
    except Exception as e:
        return f"Price Error: {e}"

def news_tool(query):
    """Searches for latest news using Tavily API."""
    if not TAVILY_API_KEY:
        return "Tavily API Key missing."
    try:
        client = TavilyClient(api_key=TAVILY_API_KEY)
        response = client.search(query=query, search_depth="basic")
        results = [f"- {r['title']}: {r['content'][:200]}..." for r in response.get('results', [])]
        return "\n".join(results[:3]) if results else "No news found."
    except Exception as e:
        return f"Search Error: {e}"

# Tool mapping
TOOLS = {
    "arithmetic": arithmetic_tool,
    "get_price": price_tool,
    "get_news": news_tool
}

## 4. Agent Implementation

In [ ]:
def router_prompt(query):
    # Double curly braces {{ }} are used to escape them in f-strings
    return f"""You are a strict AI Workflow Router. 
Analyze the user query and output a JSON array of tool calls.

TOOLS:
1. arithmetic(op, a, b): op=['add','subtract','multiply','divide']. 
   Note: If a value is unknown (like a stock price), do NOT call arithmetic yet.
2. get_price(symbol): symbol is stock (NVDA, AAPL) or crypto (BTC-USD, ETH-USD).
3. get_news(query): news search string.

RULES:
- Output ONLY a valid JSON array. 
- Do NOT include markdown code blocks (```json).
- Do NOT include explanations.
- Handle multiple tasks simultaneously.
- For math involving real-time prices (e.g., \"Price of BTC + 500\"), ONLY call 'get_price'. The Synthesizer will handle the math.

Example Query: \"Calculate 1024 times 4 and show BTC price\"
Example Output: [{{\"function\": \"arithmetic\", \"args\": {{\"op\": \"multiply\", \"a\": 1024, \"b\": 4}}}}, {{\"function\": \"get_price\", \"args\": {{\"symbol\": \"BTC-USD\"}}}}]

User Query: {query}
JSON:
"""

def synthesize_prompt(query, metadata):
    return f"""You are a Financial Research Assistant Assistant.
Combine the user query and the retrieved tool results into a clear, professional answer.
If the tool results contain a price, and the user asked for math (e.g., + 500), PERFORM that calculation yourself in the final answer.

User Query: {query}
Tool Results: {json.dumps(metadata, indent=2)}

Final Answer:
"""

def extract_json(text):
    """Robustly extracts JSON from potentially messy model output."""
    t = text.strip()
    # Remove logic for <think> tags as Instruct model typically doesn't use them
    if "</think>" in t:
        t = t.split("</think>")[-1].strip()
    if t.startswith("```json"):
        t = t[7:].strip()
    if t.startswith("```"):
        t = t[3:].strip()
    if t.endswith("```"):
        t = t[:-3].strip()
    
    start = t.find("[")
    end = t.rfind("]")
    if start != -1 and end != -1:
        return t[start:end+1]
    return t

class MultiStepAgent:
    def __init__(self):
        pass

    def run(self, query):
        print(f"[*] Routing: {query}")
        router_out_raw = generate(router_prompt(query))
        router_out = extract_json(router_out_raw)
        
        try:
            tool_calls = json.loads(router_out)
        except Exception as e:
            return f"Router Error: Could not parse JSON.\nRaw Output: {router_out_raw}\nError: {e}"

        results = []
        for call in tool_calls:
            func_name = call.get("function")
            args = call.get("args", {})
            print(f"[*] Executing: {func_name}({args})")
            
            if func_name in TOOLS:
                res = TOOLS[func_name](**args)
                results.append({"tool": func_name, "args": args, "result": res})
            else:
                results.append({"tool": func_name, "error": "Function not found"})

        print(f"[*] Synthesizing final answer...")
        final_answer = generate(synthesize_prompt(query, results))
        # Clean up any potential leftover thinking (robustness)
        if "</think>" in final_answer:
            final_answer = final_answer.split("</think>")[-1].strip()
        
        return final_answer

## 5. Testing the Agent

In [ ]:
agent = MultiStepAgent()

queries = [
    "Calculate 1024 times 4 and give me the latest Bitcoin price.",
    "What is the latest news for Nvidia and what is their current stock price?",
    "If I take the current Ethereum price and add 500, what is the result?"
]

for q in queries:
    print("="*50)
    print(f"USER: {q}")
    response = agent.run(q)
    print(f"AGENT: {response}")

USER: Calculate 1024 times 4 and give me the latest Bitcoin price.
[*] Routing: Calculate 1024 times 4 and give me the latest Bitcoin price.
[*] Executing: arithmetic({'op': 'multiply', 'a': 1024, 'b': 4})
[*] Executing: get_price({'symbol': 'BTC-USD'})
[*] Synthesizing final answer...
AGENT: The product of 1024 and 4 is **4096**.  
The latest Bitcoin price is approximately **$88,259.20**.

Let me know if you need further analysis!
USER: What is the latest news for Nvidia and what is their current stock price?
[*] Routing: What is the latest news for Nvidia and what is their current stock price?
[*] Executing: get_news({'query': 'latest news for Nvidia'})
[*] Executing: get_price({'symbol': 'NVDA'})
[*] Synthesizing final answer...
AGENT: The latest news for Nvidia could not be retrieved due to an API key issue. However, the current stock price for NVDA is approximately $186.47. Let me know if you'd like further analysis or updates.
USER: If I take the current Ethereum price and add 50